In [1]:
import os
import pandas as pd

In [2]:
# Obtendo os produtos de interesse agropecuário:

isic = pd.read_csv('DB/NCM_ISIC.csv', encoding='latin', sep = ';')
isic_agro = isic.loc[isic['CO_ISIC_SECAO'] == 'A']

isic_agro = isic_agro[['CO_ISIC_CLASSE', 'NO_ISIC_CLASSE', 'NO_ISIC_GRUPO', 
                       'NO_ISIC_DIVISAO']]

ncm = pd.read_csv('DB/NCM.csv', encoding='latin', sep = ';')
ncm_agro = pd.merge(isic_agro, ncm, on = 'CO_ISIC_CLASSE', how = 'inner')

ncm_agro.drop(['CO_ISIC_CLASSE', 'CO_UNID', 'CO_SH6', 'CO_PPE', 'CO_PPI', 
       'CO_FAT_AGREG', 'CO_CUCI_ITEM', 'CO_CGCE_N3', 'CO_SIIT', 'CO_EXP_SUBSET', 
       'NO_NCM_ESP', 'NO_NCM_ING'], axis = 1, inplace=True)

ncm_agro.head(2)

,NO_ISIC_CLASSE,NO_ISIC_GRUPO,NO_ISIC_DIVISAO,CO_NCM,NO_NCM_POR
0,Cultivo de outras culturas perenes,Cultivo de culturas perenes,"Produção vegetal, animal e caça",14011000,Bambus para cestaria ou espartaria
1,Cultivo de outras culturas perenes,Cultivo de culturas perenes,"Produção vegetal, animal e caça",14012000,Rotins para cestaria ou espartaria


In [3]:
# Filtrando os produtos agropecuários exportados

exp = pd.DataFrame()

for file in os.listdir('DB/EXP/'):
    data = pd.read_csv('DB/EXP/'+ file, encoding='latin', sep=';')
    data = data.loc[data['CO_NCM'].isin(ncm_agro['CO_NCM'])]
    exp = pd.concat([exp, data], ignore_index=True)
exp_agro = pd.merge(exp, ncm_agro, on = 'CO_NCM', how = 'inner')

In [ ]:
# Filtrando os produtos agropecuários importados

imp = pd.DataFrame()

for file in os.listdir('DB/IMP/'):
    data = pd.read_csv('DB/IMP/'+ file, encoding='latin', sep=';')
    data = data.loc[data['CO_NCM'].isin(ncm_agro['CO_NCM'])]
    imp = pd.concat([imp, data], ignore_index=True)
imp_agro = pd.merge(imp, ncm_agro, on = 'CO_NCM', how = 'inner')

In [ ]:
pais = pd.read_csv('DB/PAIS.csv', encoding='latin', sep= ';')

exp_agro = pd.merge(exp_agro, pais[['CO_PAIS', 'NO_PAIS']], on='CO_PAIS', how = 'left')
exp_agro.drop('CO_PAIS', axis=1, inplace=True)

imp_agro = pd.merge(imp_agro, pais[['CO_PAIS', 'NO_PAIS']], on='CO_PAIS', how = 'left')
imp_agro.drop('CO_PAIS', axis=1, inplace=True)

In [ ]:
via = pd.read_csv('DB/VIA.csv', encoding = 'latin', sep = ';')

exp_agro = pd.merge(exp_agro, via[['CO_VIA', 'NO_VIA']], on='CO_VIA', how = 'left')
exp_agro.drop('CO_VIA', axis=1, inplace=True)

imp_agro = pd.merge(imp_agro, via[['CO_VIA', 'NO_VIA']], on='CO_VIA', how = 'left')
imp_agro.drop('CO_VIA', axis=1, inplace=True)

In [ ]:
uf = pd.read_csv('DB/UF.csv', encoding= 'latin', sep=';')

exp_agro = pd.merge(exp_agro, uf[['SG_UF', 'NO_UF', 'NO_REGIAO']], 
                    left_on='SG_UF_NCM', right_on='SG_UF', how = 'left')
exp_agro.drop('SG_UF_NCM', axis=1, inplace=True)

imp_agro = pd.merge(imp_agro, uf[['SG_UF', 'NO_UF', 'NO_REGIAO']], 
                    left_on='SG_UF_NCM', right_on='SG_UF', how = 'left')
imp_agro.drop('SG_UF_NCM', axis=1, inplace=True)


In [ ]:
exp_agro.drop(['CO_NCM', 'CO_UNID', 'CO_URF', 'QT_ESTAT'], axis = 1, inplace=True)

imp_agro.drop(['CO_NCM', 'CO_UNID', 'CO_URF', 'QT_ESTAT'], axis = 1, inplace=True)

In [ ]:
exp_agro['Data'] = pd.to_datetime(exp_agro['CO_ANO'].astype(str) + '-' + exp_agro['CO_MES'].astype(str), format='%Y-%m')
exp_agro.drop(['CO_ANO','CO_MES'], axis = 1, inplace=True)

imp_agro['Data'] = pd.to_datetime(imp_agro['CO_ANO'].astype(str) + '-' + imp_agro['CO_MES'].astype(str), format='%Y-%m')
imp_agro.drop(['CO_ANO','CO_MES'], axis = 1, inplace=True)

In [ ]:
# Exportando os dados
exp_agro.to_csv('DB/exp_agro.csv')
imp_agro.to_csv('DB/imp_agro.csv')